In [1]:
start_date_query='2024-01-01'
end_date_query=None
# end_date_query='2023-12-24' 

# def create_incident_json_data(start_date_query:str,end_date_query:str,config)-> list[str] :
# """
# Retrive incident data from Database to create json file and save them in local directory.
# Args:
#     start_date_query (str): Start date to get data.
#     end_date_query (str): End date to get data.
#     config : .env Config file
# Returns:
#     list[str]: list of newly created json files to be imported to google cloud storage.
# """

In [4]:
import psycopg2
import psycopg2.extras as extras
import pandas as pd
import json
from datetime import datetime
from google.cloud import bigquery
from dotenv import dotenv_values
import re

from google.cloud.exceptions import NotFound
from google.api_core.exceptions import BadRequest

from dotenv import dotenv_values

In [5]:
env_path='.env'
config = dotenv_values(dotenv_path=env_path)

local_data_path=config['INPUT_SEARCH_DATA_PATH']
url_search_detail=config['SEARCH_DETAIL_URL']
customDocumentId=config['CUSTOM_DOCUMENT_ID']

str_postfix="daily_incident"

listNewlyCratedFiles=[]

In [6]:
dt_imported=datetime.now()
str_imported=dt_imported.strftime('%Y-%m-%d %H:%M:%S')
str_date_imported=dt_imported.strftime('%d%m%Y_%H%M')
print(f"Imported DateTime: {str_imported}" )
print(f"Imported Date: {str_date_imported}" )

Imported DateTime: 2024-02-24 17:25:20
Imported Date: 24022024_1725


In [7]:
def get_postgres_conn():
 try:
  conn = psycopg2.connect(
        database=config['DATABASES_NAME'], user=config['DATABASES_USER'],
      password=config['DATABASES_PASSWORD'], host=config['DATABASES_HOST']
     )
  return conn

 except Exception as error:
  print(error)      
  raise error
def list_data(sql,params,connection):
 df=None   
 with connection.cursor() as cursor:
    
    if params is None:
       cursor.execute(sql)
    else:
       cursor.execute(sql,params)
    
    columns = [col[0] for col in cursor.description]
    dataList = [dict(zip(columns, row)) for row in cursor.fetchall()]
    df = pd.DataFrame(data=dataList) 
 return df 

In [17]:
if end_date_query  is None:
    date_filter=f"incident.updated_at>='{start_date_query}'"
else:
    date_filter=f"incident.updated_at>='{start_date_query}' and incident.updated_at<'{end_date_query}'"

# ,CONCAT(incident_no,' - Serial: ',serial_number,' - Model:',model.model_name,' - Brand:',brand_name) as title
sql_incident=f"""

select
incident.id as _id ,incident.incident_no as incident_no,ac.company_name
,CONCAT('{url_search_detail}',incident.id,'/') as uri,

incident.incident_subject as title, incident_description description

,xtype.incident_type_name as incident_type,severity.severity_name as  severity

,CONCAT(brand.brand_name,' - ',model.model_name,'- ',serial_number) as inventory_item
,brand.brand_name as brand,model.model_name as model,inventory.serial_number
,incident.updated_at AT time zone 'utc' as updated_at

from app_incident as incident
inner join app_incident_type as  xtype on incident.incident_type_id = xtype.id
inner join  app_incident_severity as severity on  incident.incident_severity_id = severity.id
inner join app_inventory as inventory on incident.inventory_id = inventory.id
inner join app_brand as brand on inventory.brand_id = brand.id
inner join app_model as model on inventory.model_id = model.id
inner join app_project ap on inventory.project_id = ap.id
inner join app_company ac on ap.company_id = ac.id

where {date_filter}
and inventory.is_dummy=False and incident.incident_status_id<>3

order by incident.id 

"""

print(sql_incident)




select
incident.id as _id ,incident.incident_no as incident_no,ac.company_name
,CONCAT('https://essm.yipintsoi.com/search/result/',incident.id,'/') as uri,

incident.incident_subject as title, incident_description description

,xtype.incident_type_name as incident_type,severity.severity_name as  severity

,CONCAT(brand.brand_name,' - ',model.model_name,'- ',serial_number) as inventory_item
,brand.brand_name as brand,model.model_name as model,inventory.serial_number
,incident.updated_at AT time zone 'utc' as updated_at

from app_incident as incident
inner join app_incident_type as  xtype on incident.incident_type_id = xtype.id
inner join  app_incident_severity as severity on  incident.incident_severity_id = severity.id
inner join app_inventory as inventory on incident.inventory_id = inventory.id
inner join app_brand as brand on inventory.brand_id = brand.id
inner join app_model as model on inventory.model_id = model.id
inner join app_project ap on inventory.project_id = ap.id
inner jo

In [18]:
sql_detail="""
select  detail.id as detail_id, detail.incident_master_id as incident_id ,
workaround_resolution as solution
,(select service_team_name from app_serviceteam as team where team.id= service_team_id ) as engineer_team
from app_incident_detail detail
 where detail.incident_master_id =  %(incident_id_param)s 
 """
print(sql_detail)


select  detail.id as detail_id, detail.incident_master_id as incident_id ,
workaround_resolution as solution
,(select service_team_name from app_serviceteam as team where team.id= service_team_id ) as engineer_team
from app_incident_detail detail
 where detail.incident_master_id =  %(incident_id_param)s 
 


In [24]:
_ILLEGAL_CHARACTERS_RE = re.compile(r"[\000-\010]|[\013-\014]|[\016-\037]")
# \r\n\r  \r\n
# https://www.geeksforgeeks.org/python-removing-newline-character-from-string/
def replace_ILLEGAL_CHARACTERS(text):
   text_fixed = _ILLEGAL_CHARACTERS_RE.sub("", text)
   return  text_fixed  

def replace_NewLine_CHARACTERS(text):
   text_fixed= text.replace("\r\n", " ").replace("\r\n\r", " ")   
   return  text_fixed  

df_all=list_data(sql_incident,None,get_postgres_conn())
print(f"List all issues dataframe : {len(df_all)}")
if df_all.empty==True:
    print("No data to create Json files")
    # return listNewlyCratedFiles
                   

List all issues dataframe : 349


In [25]:
df_all[customDocumentId]=df_all[customDocumentId].astype(str)


df_all['description']=df_all['description'].apply(replace_ILLEGAL_CHARACTERS) 
df_all['description']=df_all['description'].apply(replace_NewLine_CHARACTERS)

df_all['updated_at']= df_all['updated_at'].dt.strftime('%Y-%m-%d %H:%M:%S')

print(df_all.info())
df_all.tail()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349 entries, 0 to 348
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   _id             349 non-null    object
 1   incident_no     349 non-null    object
 2   company_name    349 non-null    object
 3   uri             349 non-null    object
 4   title           349 non-null    object
 5   description     349 non-null    object
 6   incident_type   349 non-null    object
 7   severity        349 non-null    object
 8   inventory_item  349 non-null    object
 9   brand           349 non-null    object
 10  model           349 non-null    object
 11  serial_number   349 non-null    object
 12  updated_at      349 non-null    object
dtypes: object(13)
memory usage: 35.6+ KB
None


,_id,incident_no,company_name,uri,title,description,incident_type,severity,inventory_item,brand,model,serial_number,updated_at
344,4614,SR-ES-24-4614,IBANK,https://essm.yipintsoi.com/search/result/4614/,YIP Support Case: 8837 : สอบถามการดู capacity ...,เรียน occ sn: SGH241WYDV สอบถามการดู capacity ...,Software,Minor,HPE - DL360 Gen10- SGH241WYDV,HPE,DL360 Gen10,SGH241WYDV,2024-02-23 03:42:42
345,4615,SR-ES-24-4615,INET- (HPE Product),https://essm.yipintsoi.com/search/result/4615/,YIP Support Case: 8886 : simplivity VM ใน host...,"Dear OCC, รบกวนเปิดเคส P1 , simplivity VM ใน...",Software,Critical,HPE - ProLiant DL380 Gen10- SGH910W4FY,HPE,ProLiant DL380 Gen10,SGH910W4FY,2024-02-23 03:43:41
346,4616,SR-ES-24-4616,IBANK,https://essm.yipintsoi.com/search/result/4616/,YIP Support Case: 8939 : [ibank] ตรวจสอบช่องโห...,Dear OCC รบกวนตรวสอบช่องโหว่ VMware ของทุกโปร...,Software,Minor,VMWare - VMware vSphere 8 Enterprise Plus for ...,VMWare,VMware vSphere 8 Enterprise Plus for 1 processor,-,2024-02-23 03:49:28
347,4617,SR-ES-24-4617,SRTET,https://essm.yipintsoi.com/search/result/4617/,คุณพันช์แจ้งว่าพนักงานรหัส 30000028 พรรัมภา เจ...,คุณพันช์แจ้งว่าพนักงานรหัส 30000028 พรรัมภา เจ...,Software,Major,eUnite - eUnite Human Resource Application- eU...,eUnite,eUnite Human Resource Application,eUnite,2024-02-23 07:04:20
348,4618,SR-ES-24-4618,IBANK,https://essm.yipintsoi.com/search/result/4618/,YIP Support Case: 9046 : [ibank] Alert A160120...,Dear OCC รบกวนตรวจสอบเพิ่มเติมให้ด้วยครับ SN:...,General Incident,Major,HPE - ProLiant DL360 Gen10- SGH241WYGR,HPE,ProLiant DL360 Gen10,SGH241WYGR,2024-02-23 16:21:24


In [26]:
# write json file/dataframe
file_name=f"{str_date_imported}-{str_postfix}.ndjson"
with open(f'{local_data_path}\\{file_name}', 'w',encoding='utf8') as f:
    
    for index, srCaseItem in df_all.iterrows(): 
        id=int(srCaseItem[customDocumentId])
        srDict=srCaseItem.to_dict()

        print(f"============================{id}==============================")    
        
        df_detail = list_data(sql_detail,{"incident_id_param": id},get_postgres_conn())
        if df_detail.empty==False:

          df_detail=df_detail.drop(columns=['incident_id','detail_id'])  
          df_detail['solution']=df_detail['solution'].apply(replace_ILLEGAL_CHARACTERS)
          df_detail['solution']=df_detail['solution'].apply(replace_NewLine_CHARACTERS)  
            
          detailDict = df_detail.to_dict(orient = 'records')
          srDict['solution_list']=detailDict

        json_object = json.dumps(srDict,ensure_ascii=False)
        f.write(json_object + '\n')
            
listNewlyCratedFiles.append(file_name) 

============================1389==============================
============================2575==============================
============================2706==============================
============================3423==============================
============================3539==============================
============================3699==============================
============================3783==============================
============================3856==============================
============================3921==============================
============================3937==============================
============================3951==============================
============================4032==============================
============================4058==============================
============================4061==============================
============================4063==============================
============================4094=======================

In [ ]:
# return  listNewlyCratedFiles